<a href="https://colab.research.google.com/github/P-PRIYA-VARSHA/gdp-dashboard/blob/main/genetic_melody_harmonizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install music21 midi2audio

In [2]:
!apt install fluidsynth
!pip install music21

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icc

In [4]:
import random
from dataclasses import dataclass
import music21
from midi2audio import FluidSynth

# Google Colab-specific setup
from google.colab import files

@dataclass(frozen=True)
class MelodyData:
    """
    A data class representing the data of a melody.
    This class encapsulates the details of a melody including its notes, total
    duration, and the number of bars. The notes are represented as a list of
    tuples, with each tuple containing a pitch and its duration. The total
    duration and the number of bars are computed based on the notes provided.
    """
    notes: list
    duration: int = None  # Computed attribute
    number_of_bars: int = None  # Computed attribute

    def __post_init__(self):
        object.__setattr__(
            self, "duration", sum(duration for _, duration in self.notes)
        )
        object.__setattr__(self, "number_of_bars", self.duration // 4)


class GeneticMelodyHarmonizer:
    """
    Generates chord accompaniments for a given melody using a genetic algorithm.
    It evolves a population of chord sequences to find one that best fits the melody based on a fitness function.
    """
    def __init__(self, melody_data, chords, population_size, mutation_rate, fitness_evaluator):
        self.melody_data = melody_data
        self.chords = chords
        self.mutation_rate = mutation_rate
        self.population_size = population_size
        self.fitness_evaluator = fitness_evaluator
        self._population = []

    def generate(self, generations=1000):
        self._population = self._initialise_population()
        for _ in range(generations):
            parents = self._select_parents()
            new_population = self._create_new_population(parents)
            self._population = new_population
        return self.fitness_evaluator.get_chord_sequence_with_highest_fitness(self._population)

    def _initialise_population(self):
        return [self._generate_random_chord_sequence() for _ in range(self.population_size)]

    def _generate_random_chord_sequence(self):
        return [random.choice(self.chords) for _ in range(self.melody_data.number_of_bars)]

    def _select_parents(self):
        fitness_values = [self.fitness_evaluator.evaluate(seq) for seq in self._population]
        return random.choices(self._population, weights=fitness_values, k=self.population_size)

    def _create_new_population(self, parents):
        new_population = []
        for i in range(0, self.population_size, 2):
            child1, child2 = self._crossover(parents[i], parents[i + 1]), self._crossover(parents[i + 1], parents[i])
            child1 = self._mutate(child1)
            child2 = self._mutate(child2)
            new_population.extend([child1, child2])
        return new_population

    def _crossover(self, parent1, parent2):
        cut_index = random.randint(1, len(parent1) - 1)
        return parent1[:cut_index] + parent2[cut_index:]

    def _mutate(self, chord_sequence):
        if random.random() < self.mutation_rate:
            mutation_index = random.randint(0, len(chord_sequence) - 1)
            chord_sequence[mutation_index] = random.choice(self.chords)
        return chord_sequence


class FitnessEvaluator:
    """
    Evaluates the fitness of a chord sequence based on various musical criteria.
    """
    def __init__(self, melody_data, chord_mappings, weights, preferred_transitions):
        self.melody_data = melody_data
        self.chord_mappings = chord_mappings
        self.weights = weights
        self.preferred_transitions = preferred_transitions

    def get_chord_sequence_with_highest_fitness(self, chord_sequences):
        return max(chord_sequences, key=self.evaluate)

    def evaluate(self, chord_sequence):
        return sum(self.weights[func] * getattr(self, f"_{func}")(chord_sequence) for func in self.weights)

    def _chord_melody_congruence(self, chord_sequence):
        score, melody_index = 0, 0
        for chord in chord_sequence:
            bar_duration = 0
            while bar_duration < 4 and melody_index < len(self.melody_data.notes):
                pitch, duration = self.melody_data.notes[melody_index]
                if pitch[0] in self.chord_mappings[chord]:
                    score += duration
                bar_duration += duration
                melody_index += 1
        return score / self.melody_data.duration

    def _chord_variety(self, chord_sequence):
        unique_chords = len(set(chord_sequence))
        total_chords = len(self.chord_mappings)
        return unique_chords / total_chords

    def _harmonic_flow(self, chord_sequence):
        score = 0
        for i in range(len(chord_sequence) - 1):
            next_chord = chord_sequence[i + 1]
            if next_chord in self.preferred_transitions[chord_sequence[i]]:
                score += 1
        return score / (len(chord_sequence) - 1)

    def _functional_harmony(self, chord_sequence):
        score = 0
        if chord_sequence[0] in ["C", "Am"]:
            score += 1
        if chord_sequence[-1] in ["C"]:
            score += 1
        if "F" in chord_sequence and "G" in chord_sequence:
            score += 1
        return score / 3


def create_score(melody, chord_sequence, chord_mappings):
    score = music21.stream.Score()
    melody_part = music21.stream.Part()
    for note_name, duration in melody:
        melody_note = music21.note.Note(note_name, quarterLength=duration)
        melody_part.append(melody_note)
    chord_part = music21.stream.Part()
    current_duration = 0
    for chord_name in chord_sequence:
        chord_notes_list = chord_mappings.get(chord_name, [])
        chord_notes = music21.chord.Chord(chord_notes_list, quarterLength=4)
        chord_notes.offset = current_duration
        chord_part.append(chord_notes)
        current_duration += 4
    score.append(melody_part)
    score.append(chord_part)
    return score


def main():
    twinkle_twinkle_melody = [
        ("C5", 1), ("C5", 1), ("G5", 1), ("G5", 1), ("A5", 1), ("A5", 1), ("G5", 2),
        ("F5", 1), ("F5", 1), ("E5", 1), ("E5", 1), ("D5", 1), ("D5", 1), ("C5", 2),
        ("G5", 1), ("G5", 1), ("F5", 1), ("F5", 1), ("E5", 1), ("E5", 1), ("D5", 2),
        ("G5", 1), ("G5", 1), ("F5", 1), ("F5", 1), ("E5", 1), ("E5", 1), ("D5", 2),
        ("C5", 1), ("C5", 1), ("G5", 1), ("G5", 1), ("A5", 1), ("A5", 1), ("G5", 2),
        ("F5", 1), ("F5", 1), ("E5", 1), ("E5", 1), ("D5", 1), ("D5", 1), ("C5", 2)
    ]

    weights = {"chord_melody_congruence": 0.4, "chord_variety": 0.1, "harmonic_flow": 0.3, "functional_harmony": 0.2}
    chord_mappings = {"C": ["C", "E", "G"], "Dm": ["D", "F", "A"], "Em": ["E", "G", "B"], "F": ["F", "A", "C"],
                      "G": ["G", "B", "D"], "Am": ["A", "C", "E"], "Bdim": ["B", "D", "F"]}
    preferred_transitions = {"C": ["G", "Am", "F"], "Dm": ["G", "Am"], "Em": ["Am", "F", "C"], "F": ["C", "G"],
                             "G": ["Am", "C"], "Am": ["Dm", "Em", "F", "C"], "Bdim": ["F", "Am"]}

    melody_data = MelodyData(twinkle_twinkle_melody)
    fitness_evaluator = FitnessEvaluator(melody_data, chord_mappings, weights, preferred_transitions)
    harmonizer = GeneticMelodyHarmonizer(melody_data, list(chord_mappings.keys()), 100, 0.2, fitness_evaluator)
    best_chord_sequence = harmonizer.generate(generations=100)

    score = create_score(twinkle_twinkle_melody, best_chord_sequence, chord_mappings)

    # Export to MusicXML and MIDI
    score.write("musicxml", fp="harmonized_melody.xml")
    score.write("midi", fp="harmonized_melody.mid")

    # Convert MIDI to WAV using FluidSynth
    fs = FluidSynth()
    fs.midi_to_audio('harmonized_melody.mid', 'harmonized_melody.wav')

    # Download the files in Colab
    files.download('harmonized_melody.xml')
    files.download('harmonized_melody.wav')


if __name__ == "__main__":
    main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>